In [290]:
from faker import Faker
from datetime import timedelta, datetime
import pandas as pd
import numpy as np
import json
import os


## Inventory Data

In [2]:
inventory_df = pd.read_csv('./data/inventory_data.csv')
inventory_df

,Date,Item_ID,Item_Type,Item_Name,Current_Stock,Min_Required,Max_Capacity,Unit_Cost,Avg_Usage_Per_Day,Restock_Lead_Time,Vendor_ID
0,2024-10-01,105,Consumable,Ventilator,1542,264,1018,4467.55,108,17,V001
1,2024-10-02,100,Equipment,Ventilator,2487,656,3556,5832.29,55,12,V001
2,2024-10-03,103,Equipment,Surgical Mask,2371,384,5562,16062.98,470,6,V001
3,2024-10-04,103,Consumable,Surgical Mask,2038,438,1131,744.10,207,15,V002
4,2024-10-05,107,Equipment,IV Drip,2410,338,1013,15426.53,158,12,V003
...,...,...,...,...,...,...,...,...,...,...,...
495,2026-02-08,108,Consumable,Ventilator,4072,776,3496,15934.20,257,6,V002
496,2026-02-09,100,Equipment,X-ray Machine,997,239,2186,5863.88,108,16,V002
497,2026-02-10,107,Equipment,Surgical Mask,4193,304,4213,90.30,419,27,V001
498,2026-02-11,106,Consumable,IV Drip,822,186,2385,5284.45,382,6,V003


In [3]:
## First, generate unique items

supplies_dict = {
    'Ventilator': {
        'item_id': 100,
        'item_type': 'Equipment',
        'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
        'unit_cost': (1500000, 3200000),
        'min_required': (3, 15),
        'max_capacity': (25, 50),
        'avg_usage_per_day': (1, 5),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
        'storage_location': 'ICU Equipment Room',
    },
    'X-Ray-Machine': {
        'item_id': 101,
        'item_type': 'Equipment',
        'description': 'A diagnostic tool that uses radiation to create images of bones and internal structures.',
        'unit_cost': (2500000, 5000000),
        'min_required': (1, 5),
        'max_capacity': (10, 20),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (20, 45),
        'expiration_days': None,
        'storage_location': 'Radiology Imaging Suite',
    },
    'Defibrilator': {
        'item_id': 102,
        'item_type': 'Equipment',
        'description': 'A life-saving device that delivers electric shocks to restore normal heart rhythms in cardiac arrest patients.',
        'unit_cost': (300000, 750000),
        'min_required': (1, 5),
        'max_capacity': (8, 15),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (20, 40),
        'expiration_days': None,
        'storage_location': 'Emergency Equipment Storage',
    },
    'Wheelchair': {
        'item_id': 103,
        'item_type': 'Equipment',
        'description': 'A mobility aid designed to assist patients with limited or no ability to walk.',
        'unit_cost': (5000, 25000), # Manual vs Electric Wheelchairs
        'min_required': (5, 20),
        'max_capacity': (30, 100),
        'avg_usage_per_day': (0, 5),
        'restock_lead_time': (7, 20),
        'expiration_days': None,
        'storage_location': 'Ward Equipment Room',
    },
    'Hospital Bed': {
        'item_id': 104,
        'item_type': 'Equipment',
        'description': 'An adjustable bed designed for patient care, enhancing comfort and accessibility in medical settings.',
        'unit_cost': (12000, 60000),
        'min_required': (25, 100),
        'max_capacity': (150, 300),
        'avg_usage_per_day': (0, 3),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
        'storage_location': 'General Equipment Storage',
    },
    'ECG Machine': {
        'item_id': 105,
        'item_type': 'Equipment',
        'description': 'A device that records the heart’s electrical activity to detect irregularities and cardiac conditions.',
        'unit_cost': (100000, 400000),
        'min_required': (2, 5),
        'max_capacity': (10, 15),
        'avg_usage_per_day': (0, 3),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
        'storage_location': 'Cardiology Equipment Room',
    },
    'MRI Scanner': {
        'item_id': 106,
        'item_type': 'Equipment',
        'description': 'An imaging system that uses magnetic fields and radio waves to create detailed images of internal organs.',
        'unit_cost': (25000000, 50000000),
        'min_required': (1, 2),
        'max_capacity': (2, 5),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (30, 60),
        'expiration_days': None,
        'storage_location': 'Radiology Imaging Suite',
    },
    'Infusion Pump': {
        'item_id': 107,
        'item_type': 'Equipment',
        'description': 'A device that precisely delivers fluids, medications, or nutrients into a patient’s bloodstream.',
        'unit_cost': (80000, 250000),
        'min_required': (3, 10),
        'max_capacity': (15, 30),
        'avg_usage_per_day': (5, 15),
        'restock_lead_time': (10, 20),
        'expiration_days': None,
        'storage_location': 'ICU Equipment Room',
    },
    'Gloves': {
        'item_id': 200,
        'item_type': 'Consumable',
        'description': 'Protective hand coverings used by medical staff to maintain hygiene and prevent contamination.',
        'unit_cost': (5, 15),
        'min_required': (1000, 3000),
        'max_capacity': (5000, 10000),
        'avg_usage_per_day': (200, 1000),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'IV Drip': {
        'item_id': 201,
        'item_type': 'Consumable',
        'description': 'A method of delivering fluids, nutrients, or medications directly into a patient\'s veins.',
        'unit_cost': (120, 350),
        'min_required': (300, 1000),
        'max_capacity': (2000, 5000),
        'avg_usage_per_day': (50, 200),
        'restock_lead_time': (5, 10),
        'expiration_days': 1825,
        'storage_location': 'Pharmacy Storage',
    },
    'Surgical Mask': {
        'item_id': 202,
        'item_type': 'Consumable',
        'description': 'A disposable face mask that protects against airborne contaminants and reduces infection spread.',
        'unit_cost': (3, 8),
        'min_required': (3000, 10000),
        'max_capacity': (15000, 50000),
        'avg_usage_per_day': (500, 3000),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'Gown': {
        'item_id': 203,
        'item_type': 'Consumable',
        'description': 'A protective garment worn by medical staff to prevent contamination and ensure a sterile environment.',
        'unit_cost': (80, 250),
        'min_required': (500, 2000),
        'max_capacity': (3000, 8000),
        'avg_usage_per_day': (100, 500),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'Face Shield': {
        'item_id': 204,
        'item_type': 'Consumable',
        'description': 'A transparent protective visor that guards against splashes, droplets, and airborne particles.',
        'unit_cost': (50, 120),
        'min_required': (500, 2000),
        'max_capacity': (4000, 10000),
        'avg_usage_per_day': (100, 800),
        'restock_lead_time': (3, 8),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'Syringe': {
        'item_id': 205,
        'item_type': 'Consumable',
        'description': 'A medical instrument used to inject or withdraw fluids from the body.',
        'unit_cost': (7, 35),
        'min_required': (1500, 5000),
        'max_capacity': (8000, 20000),
        'avg_usage_per_day': (200, 2000),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
        'storage_location': 'Pharmacy Storage',
    },
    'Bandages': {
        'item_id': 206,
        'item_type': 'Consumable',
        'description': 'Sterile materials used to cover wounds and promote healing while preventing infections.',
        'unit_cost': (10, 50),
        'min_required': (1000, 4000),
        'max_capacity': (6000, 12000),
        'avg_usage_per_day': (300, 1500),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'Blood Bags': {
        'item_id': 207,
        'item_type': 'Consumable',
        'description': 'Special containers designed for the collection, storage, and transfusion of blood.',
        'unit_cost': (400, 1500),
        'min_required': (300, 1000),
        'max_capacity': (1500, 5000),
        'avg_usage_per_day': (20, 150),
        'restock_lead_time': (5, 15),
        'expiration_days': 42,
        'storage_location': 'Emergency Consumables Depot',
    },
    'Antiseptic Solution': {
        'item_id': 208,
        'item_type': 'Consumable',
        'description': 'A liquid disinfectant used to cleanse wounds and prevent infections.',
        'unit_cost': (200, 700),
        'min_required': (500, 2000),
        'max_capacity': (3000, 8000),
        'avg_usage_per_day': (100, 500),
        'restock_lead_time': (5, 12),
        'expiration_days': 1095,
        'storage_location': 'Emergency Consumables Depot',
    },
    'Cotton Rolls': {
        'item_id': 209,
        'item_type': 'Consumable',
        'description': 'Soft, absorbent materials used in medical applications for wound care and cleaning.',
        'unit_cost': (50, 150),
        'min_required': (300, 1000),
        'max_capacity': (2000, 6000),
        'avg_usage_per_day': (50, 300),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
}

In [4]:
item_var_dict = {}
fake = Faker()
supplier_names = [fake.company() + ' Medical Supplies' for _ in range(5)]
contact_numbers = [fake.numerify("+63 9## ### ####") for _ in range(5)]

for key, value in supplies_dict.items():
    # Generate 5 different item variations for 5 different vendors
    item_name = key
    item_id = value['item_id']
    item_type = value['item_type']
    description = value['description']
    status = 'Active'
    min_required = value['min_required'][0]
    max_capacity = value['max_capacity'][0]

    for i in range(5):
        vendor_id = f'V{str(i+1).zfill(3)}'
        vendor_name = supplier_names[i]
        vendor_contact = contact_numbers[i]

        item_var_dict[f'{item_name}-{vendor_id}'] = {
            'item_name': item_name,
            'item_id': item_id,
            'item_type': item_type,
            'description': description,
            'status': status,
            'min_required': min_required,
            'max_capacity': max_capacity,
            'vendor_name': vendor_name,
            'vendor_contact': vendor_contact,
        }

item_var_dict

{'Ventilator-V001': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_capacity': 25,
  'vendor_name': 'Bell-Wright Medical Supplies',
  'vendor_contact': '+63 995 342 5169'},
 'Ventilator-V002': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_capacity': 25,
  'vendor_name': 'Frazier-Randolph Medical Supplies',
  'vendor_contact': '+63 931 599 3031'},
 'Ventilator-V003': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_capacity': 

In [5]:
fake = Faker()
date_list = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D').strftime('%Y-%m-%d').tolist()

# Data dictionary (to pass to df constructor)
date_arr = []
item_id_arr = []
item_name_arr = []
item_type_arr = []
description_arr = []
status_arr = []
stock_status_arr = []
unit_cost_arr = []
min_required_arr = []
max_capacity_arr = []
current_stock_arr = []
avg_usage_per_day_arr = []
restock_lead_time_arr = []
vendor_name_arr = []
vendor_contact_arr = []
expiration_date_arr = []
created_by_arr = []

trend_factor = np.ones(len(date_list))
noise_factor = np.random.normal(1.0, 0.05, len(date_list))

def seasonal_multiplier(day_of_year):
    return 1 + 0.05 * np.sin(2 * np.pi * day_of_year / 365)  # Weak seasonality

def accept_unused_date(used_dates_set):
    while True:
        selected_date = fake.random_element(date_list)
        if selected_date not in used_dates_set:
            used_dates_set.add(selected_date)
            return selected_date

current_unique_item = ''
used_dates = set()

for key, value in item_var_dict.items():
    if current_unique_item != value['item_name']:
        current_unique_item = value['item_name']
        used_dates = set()

    # Generate around 200 - 215 random instances of a particular item
    for _ in range(np.random.randint(200, 216)):
        date = accept_unused_date(used_dates)
        month = pd.to_datetime(date).month
        date_obj = pd.to_datetime(date)
        day_of_year = date_obj.dayofyear
        time_index = date_list.index(date)
        
        item_id = value['item_id']
        item_name = value['item_name']
        item_type = value['item_type']
        description = value['description']
        status = value['status']
        stock_status = 'Available'
        unit_cost = np.random.randint(supplies_dict[item_name]['unit_cost'][0], supplies_dict[item_name]['unit_cost'][1] + 1)
        min_required = value['min_required']
        max_capacity = value['max_capacity']

        base_stock = np.random.randint(value['min_required'], value['max_capacity'] + 1)
        stock_with_trend = base_stock * trend_factor[time_index]  # No trend
        stock_seasonal = stock_with_trend * seasonal_multiplier(day_of_year)  # No seasonality
        stock_randomized = stock_seasonal * noise_factor[time_index]  # Small random noise
        current_stock = int(np.clip(stock_randomized, value['min_required'], value['max_capacity']))  # Keep within limits

        avg_usage_per_day = np.random.randint(supplies_dict[item_name]['avg_usage_per_day'][0], supplies_dict[item_name]['avg_usage_per_day'][1] + 1)
        restock_lead_time = np.random.randint(supplies_dict[item_name]['restock_lead_time'][0], supplies_dict[item_name]['restock_lead_time'][1] + 1)
        vendor_name = value['vendor_name']
        vendor_contact = value['vendor_contact']
        created_by = '67b1fdb293bfc8e17f896b31'

        if item_type == 'Consumable':
            expiration_duration = supplies_dict[item_name]['expiration_days']
            expiration_date = pd.to_datetime(date) + timedelta(days=expiration_duration)
        else:
            expiration_date = pd.NaT

        date_arr.append(date)
        item_id_arr.append(item_id)
        item_name_arr.append(item_name)
        item_type_arr.append(item_type)
        description_arr.append(description)
        status_arr.append(status)
        stock_status_arr.append(stock_status)
        unit_cost_arr.append(unit_cost)
        min_required_arr.append(min_required)
        max_capacity_arr.append(max_capacity)
        current_stock_arr.append(current_stock)
        avg_usage_per_day_arr.append(avg_usage_per_day)
        restock_lead_time_arr.append(restock_lead_time)
        vendor_name_arr.append(vendor_name)
        vendor_contact_arr.append(vendor_contact)
        expiration_date_arr.append(expiration_date)
        created_by_arr.append(created_by)

data = list(zip(date_arr, item_id_arr, item_name_arr, item_type_arr, description_arr, status_arr, stock_status_arr, unit_cost_arr, min_required_arr, max_capacity_arr, 
                current_stock_arr, avg_usage_per_day_arr, restock_lead_time_arr, vendor_name_arr, vendor_contact_arr, expiration_date_arr, created_by_arr))

df = pd.DataFrame(data, columns=[
    'date', 'item_id', 'item_name', 'item_type', 'description', 'status', 'stock_status', 'unit_cost', 'min_required', 'max_capacity', 'current_stock', 'avg_usage_per_day', 'restock_lead_time', 'vendor_name', 'vendor_contact', 'expiration_date', 'created_by'
]).sort_values(by=['date']).reset_index(drop=True)

## Item Listing Data

In [6]:
item_listing_df = df[['item_id', 'item_name', 'description', 'item_type', 'min_required', 'max_capacity', 'created_by', 'status']].drop_duplicates().sort_values('item_id').reset_index(drop=True)
item_listing_df

,item_id,item_name,description,item_type,min_required,max_capacity,created_by,status
0,100,Ventilator,A medical device that supports or takes over b...,Equipment,3,25,67b1fdb293bfc8e17f896b31,Active
1,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,1,10,67b1fdb293bfc8e17f896b31,Active
2,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,1,8,67b1fdb293bfc8e17f896b31,Active
3,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,5,30,67b1fdb293bfc8e17f896b31,Active
4,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,25,150,67b1fdb293bfc8e17f896b31,Active
5,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,2,10,67b1fdb293bfc8e17f896b31,Active
6,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,1,2,67b1fdb293bfc8e17f896b31,Active
7,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,3,15,67b1fdb293bfc8e17f896b31,Active
8,200,Gloves,Protective hand coverings used by medical staf...,Consumable,1000,5000,67b1fdb293bfc8e17f896b31,Active
9,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,300,2000,67b1fdb293bfc8e17f896b31,Active


In [7]:
file_name_csv = './data/item_listing_v1.csv'
file_name_json = './data/item_listing_v1.json'

if not os.path.exists(file_name_csv):
    item_listing_df.to_csv(file_name_csv, index=False)
    item_listing_df.to_json(file_name_json, orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')

File already exists!


## Item Stock Listing Data

In [8]:
item_listing_df = pd.read_json('./data/exported_listings.json')
item_listing_df

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67b5d1d3965831513a85d8be,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67b5d1d3965831513a85d8bf,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67b5d1d3965831513a85d8c0,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67b5d1d3965831513a85d8c1,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67b5d1d3965831513a85d8c2,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
5,67b5d1d3965831513a85d8c3,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,C,2,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
6,67b5d1d3965831513a85d8c4,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,C,1,2,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
7,67b5d1d3965831513a85d8c5,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,C,3,15,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
8,67b5d1d3965831513a85d8c6,200,Gloves,Protective hand coverings used by medical staf...,Consumable,C,1000,5000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
9,67b5d1d3965831513a85d8c7,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,C,300,2000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [9]:
df

,date,item_id,item_name,item_type,description,status,stock_status,unit_cost,min_required,max_capacity,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,created_by
0,2022-01-01,104,Hospital Bed,Equipment,"An adjustable bed designed for patient care, e...",Active,Available,21389,25,150,53,2,26,Sharp-Hughes Medical Supplies,+63 978 864 1649,NaT,67b1fdb293bfc8e17f896b31
1,2022-01-01,209,Cotton Rolls,Consumable,"Soft, absorbent materials used in medical appl...",Active,Available,142,300,2000,1718,236,9,Sharp-Hughes Medical Supplies,+63 978 864 1649,2026-12-31,67b1fdb293bfc8e17f896b31
2,2022-01-01,102,Defibrilator,Equipment,A life-saving device that delivers electric sh...,Active,Available,473741,1,8,5,1,38,Sharp-Hughes Medical Supplies,+63 978 864 1649,NaT,67b1fdb293bfc8e17f896b31
3,2022-01-01,203,Gown,Consumable,A protective garment worn by medical staff to ...,Active,Available,240,500,3000,2046,299,9,Butler-Green Medical Supplies,+63 969 011 8301,2026-12-31,67b1fdb293bfc8e17f896b31
4,2022-01-01,100,Ventilator,Equipment,A medical device that supports or takes over b...,Active,Available,2595599,3,25,11,1,15,Frazier-Randolph Medical Supplies,+63 931 599 3031,NaT,67b1fdb293bfc8e17f896b31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18688,2024-12-31,201,IV Drip,Consumable,"A method of delivering fluids, nutrients, or m...",Active,Available,321,300,2000,696,50,8,Butler-Green Medical Supplies,+63 969 011 8301,2029-12-30,67b1fdb293bfc8e17f896b31
18689,2024-12-31,100,Ventilator,Equipment,A medical device that supports or takes over b...,Active,Available,1687834,3,25,13,2,27,Herman-Gates Medical Supplies,+63 913 269 2456,NaT,67b1fdb293bfc8e17f896b31
18690,2024-12-31,207,Blood Bags,Consumable,Special containers designed for the collection...,Active,Available,854,300,1500,1226,81,14,Butler-Green Medical Supplies,+63 969 011 8301,2025-02-11,67b1fdb293bfc8e17f896b31
18691,2024-12-31,208,Antiseptic Solution,Consumable,A liquid disinfectant used to cleanse wounds a...,Active,Available,238,500,3000,1608,321,10,Sharp-Hughes Medical Supplies,+63 978 864 1649,2027-12-31,67b1fdb293bfc8e17f896b31


In [10]:
df[(df['item_name'] == 'Gloves') & (df['date'].str.startswith('2024-01'))]

,date,item_id,item_name,item_type,description,status,stock_status,unit_cost,min_required,max_capacity,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,created_by
12443,2024-01-01,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,10,1000,5000,1697,407,8,Herman-Gates Medical Supplies,+63 913 269 2456,2028-12-30,67b1fdb293bfc8e17f896b31
12474,2024-01-02,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,6,1000,5000,4119,409,10,Bell-Wright Medical Supplies,+63 995 342 5169,2028-12-31,67b1fdb293bfc8e17f896b31
12492,2024-01-03,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,13,1000,5000,4084,271,5,Sharp-Hughes Medical Supplies,+63 978 864 1649,2029-01-01,67b1fdb293bfc8e17f896b31
12511,2024-01-05,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,11,1000,5000,5000,699,5,Bell-Wright Medical Supplies,+63 995 342 5169,2029-01-03,67b1fdb293bfc8e17f896b31
12541,2024-01-06,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,9,1000,5000,2803,901,11,Frazier-Randolph Medical Supplies,+63 931 599 3031,2029-01-04,67b1fdb293bfc8e17f896b31
12556,2024-01-07,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,15,1000,5000,2110,941,11,Herman-Gates Medical Supplies,+63 913 269 2456,2029-01-05,67b1fdb293bfc8e17f896b31
12581,2024-01-08,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,11,1000,5000,1753,990,11,Bell-Wright Medical Supplies,+63 995 342 5169,2029-01-06,67b1fdb293bfc8e17f896b31
12599,2024-01-09,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,14,1000,5000,1060,299,9,Butler-Green Medical Supplies,+63 969 011 8301,2029-01-07,67b1fdb293bfc8e17f896b31
12607,2024-01-10,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,11,1000,5000,4155,662,11,Bell-Wright Medical Supplies,+63 995 342 5169,2029-01-08,67b1fdb293bfc8e17f896b31
12634,2024-01-11,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,14,1000,5000,4667,718,10,Frazier-Randolph Medical Supplies,+63 931 599 3031,2029-01-09,67b1fdb293bfc8e17f896b31


In [11]:
df_sub = df[['item_id', 'date', 'stock_status', 'unit_cost', 'current_stock', 'avg_usage_per_day', 'restock_lead_time', 'vendor_name', 'vendor_contact', 'expiration_date']]

stock_listing_df = item_listing_df[['_id', 'itemCode']].merge(df_sub, left_on='itemCode', right_on='item_id')
stock_listing_df.drop(columns=['itemCode', 'item_id'], inplace=True)
stock_listing_df.sort_values(by='date', inplace=True)
stock_listing_df

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2595599,11,1,15,Frazier-Randolph Medical Supplies,+63 931 599 3031,NaT
2071,67b5d1d3965831513a85d8c0,2022-01-01,Available,473741,5,1,38,Sharp-Hughes Medical Supplies,+63 978 864 1649,NaT
3104,67b5d1d3965831513a85d8c1,2022-01-01,Available,19566,25,4,15,Bell-Wright Medical Supplies,+63 995 342 5169,NaT
4152,67b5d1d3965831513a85d8c2,2022-01-01,Available,21389,53,2,26,Sharp-Hughes Medical Supplies,+63 978 864 1649,NaT
5192,67b5d1d3965831513a85d8c3,2022-01-01,Available,138972,10,0,20,Bell-Wright Medical Supplies,+63 995 342 5169,NaT
...,...,...,...,...,...,...,...,...,...,...
14550,67b5d1d3965831513a85d8cb,2024-12-31,Available,23,4334,294,5,Herman-Gates Medical Supplies,+63 913 269 2456,2029-12-30
4151,67b5d1d3965831513a85d8c1,2024-12-31,Available,17415,27,0,9,Butler-Green Medical Supplies,+63 969 011 8301,NaT
17651,67b5d1d3965831513a85d8ce,2024-12-31,Available,238,1608,321,10,Sharp-Hughes Medical Supplies,+63 978 864 1649,2027-12-31
16617,67b5d1d3965831513a85d8cd,2024-12-31,Available,854,1226,81,14,Butler-Green Medical Supplies,+63 969 011 8301,2025-02-11


In [12]:
file_name_json = './data/stock_listing_v1_1.json'

if not os.path.exists(file_name_json):
    stock_listing_df.to_json(file_name_json, orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')

File already exists!


## Patient Data

In [13]:
patient_df = pd.read_csv('./data/patient_data.csv')
patient_df

,Patient_ID,Admission_Date,Discharge_Date,Primary_Diagnosis,Procedure_Performed,Room_Type,Bed_Days,Supplies_Used,Equipment_Used,Staff_Needed
0,P001,2024-10-06 05:30:28,2024-10-23 01:11:34,Diabetes,Appendectomy,General Ward,2,"Gloves, IV",Surgical Table,2 Surgeons
1,P002,2024-10-24 11:07:58,2024-10-15 05:16:54,Fracture,Appendectomy,ICU,10,"Gown, IV",MRI Machine,1 Nurse
2,P003,2024-10-22 21:43:43,2024-10-24 10:56:30,Fracture,Chest X-ray,ICU,10,"Gloves, IV",X-ray Machine,"1 Nurse, 1 Doctor"
3,P004,2024-10-05 17:04:05,2024-10-30 14:10:01,Diabetes,Chest X-ray,ICU,11,"Gloves, IV",X-ray Machine,1 Nurse
4,P005,2024-10-21 17:04:00,2024-10-08 15:53:22,Appendicitis,MRI,ICU,2,"Gloves, IV",X-ray Machine,2 Surgeons
...,...,...,...,...,...,...,...,...,...,...
495,P496,2024-10-01 12:54:33,2024-10-06 14:31:39,Diabetes,MRI,ICU,13,"Mask, Gown",Surgical Table,"1 Nurse, 1 Doctor"
496,P497,2024-10-22 07:37:10,2024-10-13 12:31:40,Appendicitis,Appendectomy,ICU,14,"Gloves, IV",MRI Machine,"1 Nurse, 1 Doctor"
497,P498,2024-10-28 03:12:20,2024-10-10 08:11:18,Pneumonia,MRI,General Ward,13,"Mask, Gown",X-ray Machine,2 Surgeons
498,P499,2024-10-04 22:02:01,2024-10-12 03:45:09,Diabetes,Chest X-ray,General Ward,1,"Gown, IV",Surgical Table,"1 Nurse, 1 Doctor"


In [14]:
# Supplies Used Item Mapping
patient_supplies_dict = {}

for i in patient_df['Supplies_Used']:
    item_arr = i.split(', ')
    for item in item_arr:
        if item not in patient_supplies_dict:
            patient_supplies_dict[item] = 1
        elif item in patient_supplies_dict:
            patient_supplies_dict[item] += 1
        
patient_supplies_dict

{'Gloves': 169, 'IV': 339, 'Gown': 331, 'Mask': 161}

In [15]:
# Equipment Used Item Mapping
equipment_dict = {}

for i in patient_df['Equipment_Used']:
    item_arr = i.split(', ')
    for item in item_arr:
        if item not in equipment_dict:
            equipment_dict[item] = 1
        elif item in equipment_dict:
            equipment_dict[item] += 1
        
equipment_dict

{'Surgical Table': 166, 'MRI Machine': 161, 'X-ray Machine': 173}

In [16]:
# Rename the keys in the dictionary to standardized item names
try:
    patient_supplies_dict['IV Drip'] = patient_supplies_dict.pop('IV')
    patient_supplies_dict['Surgical Mask'] = patient_supplies_dict.pop('Mask')
    equipment_dict['X-Ray-Machine'] = equipment_dict.pop('X-ray Machine')
    equipment_dict['MRI Scanner'] = equipment_dict.pop('MRI Machine')
except KeyError:
    pass

In [17]:
# EDA of 'primary_diagnosis'
patient_df['Primary_Diagnosis'].value_counts()

Primary_Diagnosis
Fracture        133
Diabetes        131
Appendicitis    120
Pneumonia       116
Name: count, dtype: int64

In [18]:
# EDA of 'procedure_performed'
patient_df['Procedure_Performed'].value_counts()

Procedure_Performed
Chest X-ray     138
Appendectomy    131
MRI             131
Blood Test      100
Name: count, dtype: int64

### Synthetic Patient Data Generation

In [19]:
diagnosis_dict = {
    'Pneumonia': {
        'Chest X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': ['Infusion Pump']
        },
        'Oxygen Therapy': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves'],
            'equipment_used': ['Ventilator']
        }
    },
    'Appendicitis': {
        'Appendectomy': {
            'supplies_used': ['Gloves', 'Gown', 'Surgical Mask', 'Antiseptic Solution', 'Bandages', 'IV Drip'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        }
    },
    'Fracture': {
        'X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        },
        'Casting': {
            'supplies_used': ['Bandages', 'Cotton Rolls', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': []
        },
        'Pain Management': {
            'supplies_used': ['Syringe', 'IV Drip', 'Gloves'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        }
    },
    'Myocardial Infarction': {
        'ECG': {
            'supplies_used': ['Gloves'],
            'equipment_used': ['ECG Machine']
        },
        'Defibrillation': {
            'supplies_used': ['Gloves', 'Surgical Mask', 'Antiseptic Solution'],
            'equipment_used': ['Defibrillator']
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Blood Bags'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Diabetes Complications': {
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Blood Bags'],
            'equipment_used': ['Infusion Pump']
        },
        'Wound Dressing': {
            'supplies_used': ['Bandages', 'Gloves', 'Antiseptic Solution', 'Cotton Rolls'],
            'equipment_used': []
        }
    },
    'Stroke': {
        'MRI': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['MRI Scanner']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'Physical Therapy': {
            'supplies_used': ['Gloves', 'Face Shield', 'Gown'],
            'equipment_used': ['Hospital Bed', 'Wheelchair']
        }
    },
    'Sepsis': {
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves', 'Blood Bags'],
            'equipment_used': []
        },
        'IV Antibiotic Therapy': {
            'supplies_used': ['IV Drip', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Severe Burns': {
        'Wound Cleaning & Dressing': {
            'supplies_used': ['Antiseptic Solution', 'Bandages', 'Gloves', 'Cotton Rolls'],
            'equipment_used': []
        },
        'Skin Grafting Surgery': {
            'supplies_used': ['Gown', 'Surgical Mask', 'Gloves'],
            'equipment_used': ['Hospital Bed']
        },
        'IV Fluid Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Traumatic Brain Injury': {
        'MRI': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['MRI Scanner']
        },
        'Ventilation Support': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves'],
            'equipment_used': ['Ventilator']
        },
        'Pain Management': {
            'supplies_used': ['Syringe', 'IV Drip', 'Gloves'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        }
    },
    'COVID-19': {
        'Oxygen Therapy': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves', 'Gown'],
            'equipment_used': ['Ventilator', 'Hospital Bed']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'Chest X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        }
    }
}

In [20]:
diagnosis_id_arr = []
date_arr = []
primary_diagnosis_arr = []
procedure_performed_arr = []
item_used_arr = []
item_type_arr = []

diagnosis_id_counter = 1

date_list = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D').strftime('%Y-%m-%d').tolist()

for diagnosis in diagnosis_dict:
    for _ in range(np.random.randint(100, 500)):
        diagnosis_id = diagnosis_id_counter
        for procedure in diagnosis_dict[diagnosis]:
            for item in diagnosis_dict[diagnosis][procedure]['supplies_used']:
                diagnosis_id_arr.append(str(diagnosis_id).zfill(5))
                date_arr.append(fake.random_element(date_list))
                primary_diagnosis_arr.append(diagnosis)
                procedure_performed_arr.append(procedure)
                item_used_arr.append(item)
                item_type_arr.append('Consumable')
            for item in diagnosis_dict[diagnosis][procedure]['equipment_used']:
                diagnosis_id_arr.append(str(diagnosis_id).zfill(5))
                date_arr.append(fake.random_element(date_list))
                primary_diagnosis_arr.append(diagnosis)
                procedure_performed_arr.append(procedure)
                item_used_arr.append(item)
                item_type_arr.append('Equipment')
        diagnosis_id_counter += 1

In [21]:
print(len(diagnosis_id_arr))
print(len(primary_diagnosis_arr))
print(len(procedure_performed_arr))
print(len(item_used_arr))
print(len(item_type_arr))

40092
40092
40092
40092
40092


In [22]:
# Data dictionary (to pass to df constructor)
data = {
    'diagnosis_id': diagnosis_id_arr,
    'date': date_arr,
    'primary_diagnosis': primary_diagnosis_arr,
    'procedure_performed': procedure_performed_arr,
    'item_used': item_used_arr,
    'item_type': item_type_arr
}

patient_df_v2 = pd.DataFrame(data)
patient_df_v2

,diagnosis_id,date,primary_diagnosis,procedure_performed,item_used,item_type
0,00001,2024-01-30,Pneumonia,Chest X-Ray,Surgical Mask,Consumable
1,00001,2024-09-10,Pneumonia,Chest X-Ray,Gloves,Consumable
2,00001,2024-12-22,Pneumonia,Chest X-Ray,Gown,Consumable
3,00001,2024-06-26,Pneumonia,Chest X-Ray,Face Shield,Consumable
4,00001,2024-04-07,Pneumonia,Chest X-Ray,X-Ray Machine,Equipment
...,...,...,...,...,...,...
40087,03171,2024-08-28,COVID-19,Chest X-Ray,Surgical Mask,Consumable
40088,03171,2024-06-20,COVID-19,Chest X-Ray,Gloves,Consumable
40089,03171,2024-08-30,COVID-19,Chest X-Ray,Gown,Consumable
40090,03171,2024-05-16,COVID-19,Chest X-Ray,Face Shield,Consumable


In [23]:
file_name = './data/patient_data_v2.csv'

if not os.path.exists(file_name):
    patient_df_v2.to_csv(file_name, index=False)
    print('File saved successfully!')
else:
    print('File already exists!')

File already exists!


# Stock Listing V2
Add 'storageLocation' column to data

In [24]:
stock_listing_df1 = pd.read_json('./data/stock_listing_v1_1.json')
stock_listing_df1 = stock_listing_df1.merge(item_listing_df[['_id', 'title']], on='_id')
stock_listing_df1.head()

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,title
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,26,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,Ventilator
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,29,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Defibrilator
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,17,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Wheelchair
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,18,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,ECG Machine
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,37,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,MRI Scanner


In [25]:
# Map defined storageLocation to each stock listing record
stock_listing_df1 = stock_listing_df1.copy()
stock_listing_df1['storageLocation'] = stock_listing_df1['title'].map(lambda x: supplies_dict[x]['storage_location'])
stock_listing_df1

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,title,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,26,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,Ventilator,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,29,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Defibrilator,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,17,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Wheelchair,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,18,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,ECG Machine,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,37,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,MRI Scanner,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,11,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Syringe,Pharmacy Storage
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,28,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ventilator,ICU Equipment Room
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,9,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Bandages,Medical Supply Room
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,5,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Gloves,Medical Supply Room


In [26]:
# Remove title column
stock_listing_df1 = stock_listing_df1.drop(['title', 'restock_lead_time'], axis=1)
stock_listing_df1

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,vendor_name,vendor_contact,expiration_date,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room


In [27]:
# Export stock_listing_v2_1.json (used for model training only!)
if not os.path.exists('./data/stock_listing_v2_1.json'):
    stock_listing_df1.to_json('./data/stock_listing_v2_1.json', orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')


File already exists!


In [28]:
# Rename ALL columns from snake case to camel case
stock_listing_df1 = stock_listing_df1.rename(columns={
    'date': 'acquisitionDate',
    '_id': 'listing',
    'current_stock': 'quantity',
    'avg_usage_per_day': 'avgUsagePerDay',
    'stock_status': 'status',
    'unit_cost': 'unitCost',
    'vendor_name': 'supplierName',
    'vendor_contact': 'supplierContact',
    'expiration_date': 'expirationDate',
})
stock_listing_df1.head(5)

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,supplierName,supplierContact,expirationDate,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite


In [29]:
# Export stock_listing_v2_2.json (to export to database)
if not os.path.exists('./data/stock_listing_v2_2.json'):
    stock_listing_df1.to_json('./data/stock_listing_v2_2.json', orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')


File already exists!


## Update Item Listing IDs in Stock Listing

In [30]:
stock_listing_df3 = pd.read_json('./data/stock_listing_v2_1.json')
stock_listing_df3

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,vendor_name,vendor_contact,expiration_date,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room


In [31]:
# Rename ALL columns from snake case to camel case
stock_listing_df3 = stock_listing_df3.rename(columns={
    'date': 'acquisitionDate',
    '_id': 'listing',
    'current_stock': 'quantity',
    'avg_usage_per_day': 'avgUsagePerDay',
    'stock_status': 'status',
    'unit_cost': 'unitCost',
    'vendor_name': 'supplierName',
    'vendor_contact': 'supplierContact',
    'expiration_date': 'expirationDate',
    }
)

In [32]:
old_item_listing = pd.read_json('./data/exported_listings.json')
old_item_listing.head()

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67b5d1d3965831513a85d8be,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67b5d1d3965831513a85d8bf,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67b5d1d3965831513a85d8c0,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67b5d1d3965831513a85d8c1,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67b5d1d3965831513a85d8c2,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [33]:
stock_listing_df3 = stock_listing_df3.merge(old_item_listing[['_id', 'title']], left_on='listing', right_on='_id')
stock_listing_df3.drop('_id', axis=1, inplace=True)
stock_listing_df3

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,supplierName,supplierContact,expirationDate,storageLocation,title
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room,Ventilator
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage,Defibrilator
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room,Wheelchair
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room,ECG Machine
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite,MRI Scanner
...,...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage,Syringe
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room,Ventilator
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room,Bandages
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room,Gloves


In [ ]:
new_item_listing = pd.read_json('./data/exported_listings_v3.json')
new_item_listing.head()

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67c01bc13619256e0fee3511,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67c01bc13619256e0fee3512,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67c01bc13619256e0fee3513,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67c01bc13619256e0fee3514,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67c01bc13619256e0fee3515,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [35]:
new_item_map = new_item_listing.set_index('title')['_id'].to_dict()
new_item_map

{'Ventilator': '67c01bc13619256e0fee3511',
 'X-Ray-Machine': '67c01bc13619256e0fee3512',
 'Defibrilator': '67c01bc13619256e0fee3513',
 'Wheelchair': '67c01bc13619256e0fee3514',
 'Hospital Bed': '67c01bc13619256e0fee3515',
 'ECG Machine': '67c01bc13619256e0fee3516',
 'MRI Scanner': '67c01bc13619256e0fee3517',
 'Infusion Pump': '67c01bc13619256e0fee3518',
 'Gloves': '67c01bc13619256e0fee3519',
 'IV Drip': '67c01bc13619256e0fee351a',
 'Surgical Mask': '67c01bc13619256e0fee351b',
 'Gown': '67c01bc13619256e0fee351c',
 'Face Shield': '67c01bc13619256e0fee351d',
 'Syringe': '67c01bc13619256e0fee351e',
 'Bandages': '67c01bc13619256e0fee351f',
 'Blood Bags': '67c01bc13619256e0fee3520',
 'Antiseptic Solution': '67c01bc13619256e0fee3521',
 'Cotton Rolls': '67c01bc13619256e0fee3522'}

In [36]:
# Update _id column of stock listing
stock_listing_df3['listing'] = stock_listing_df3['title'].map(lambda x: new_item_map[x])
stock_listing_df3.drop('title', axis=1, inplace=True)
stock_listing_df3

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,supplierName,supplierContact,expirationDate,storageLocation
0,67c01bc13619256e0fee3511,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67c01bc13619256e0fee3513,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67c01bc13619256e0fee3514,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67c01bc13619256e0fee3516,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67c01bc13619256e0fee3517,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...
18643,67c01bc13619256e0fee351e,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage
18644,67c01bc13619256e0fee3511,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room
18645,67c01bc13619256e0fee351f,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room
18646,67c01bc13619256e0fee3519,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room


In [37]:
fake = Faker()
item_listing_dic = {}

def add_supplier_data(stock_data):
    listing_id = stock_data['listing']
    if listing_id not in item_listing_dic:
        supplier_name = stock_data['supplierName']
        contact_person = fake.name()
        contact_number = stock_data['supplierContact']
        email = f'{contact_person.lower().replace(' ', '_')}@gmail.com'
        manufacturer = fake.company() + ' Industries'
        
        item_listing_dic[listing_id] = {
            'supplier': {
                'name': supplier_name,
                'contactPerson': contact_person,
                'contactNumber': contact_number,
                'email': email,
            },
            'manufacturer': manufacturer,
        }

# Add manufacturerName, contactPerson, email (supplier)
stock_listing_df3.apply(add_supplier_data, axis=1)
stock_listing_df3['supplier'] = stock_listing_df3['listing'].map(lambda x: item_listing_dic[x]['supplier'])
stock_listing_df3['manufacturer'] = stock_listing_df3['listing'].map(lambda x: item_listing_dic[x]['manufacturer'])
stock_listing_df3.drop(columns=['supplierName', 'supplierContact'], inplace=True)

# Add 'note' column
stock_listing_df3['notes'] = '-'
stock_listing_df3


,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,expirationDate,storageLocation,supplier,manufacturer,notes
0,67c01bc13619256e0fee3511,2022-01-01,Available,2628329,22,5,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...",Hall and Sons Industries,-
1,67c01bc13619256e0fee3513,2022-01-01,Available,650745,6,0,NaN,Emergency Equipment Storage,"{'name': 'Miles Inc Medical Supplies', 'contac...","Tate, Chaney and Turner Industries",-
2,67c01bc13619256e0fee3514,2022-01-01,Available,15486,23,0,NaN,Ward Equipment Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...",Webb-Ward Industries,-
3,67c01bc13619256e0fee3516,2022-01-01,Available,314121,2,2,NaN,Cardiology Equipment Room,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Campbell PLC Industries,-
4,67c01bc13619256e0fee3517,2022-01-01,Available,40522506,1,1,NaN,Radiology Imaging Suite,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Melendez, Kim and Allen Industries",-
...,...,...,...,...,...,...,...,...,...,...,...
18643,67c01bc13619256e0fee351e,2024-12-31,Available,20,5990,680,1.893283e+12,Pharmacy Storage,"{'name': 'Wang-Foster Medical Supplies', 'cont...","Hicks, Hayden and Hall Industries",-
18644,67c01bc13619256e0fee3511,2024-12-31,Available,1852964,22,1,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...",Hall and Sons Industries,-
18645,67c01bc13619256e0fee351f,2024-12-31,Available,11,3405,430,1.893283e+12,Medical Supply Room,"{'name': 'Miles Inc Medical Supplies', 'contac...",Morris-Lam Industries,-
18646,67c01bc13619256e0fee3519,2024-12-31,Available,13,4101,708,1.893283e+12,Medical Supply Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...",Wood Inc Industries,-


In [39]:
# Export stock_listing_v3.json (to export to database)
if not os.path.exists('./data/stock_listing_v3.json'):
    stock_listing_df3.to_json('./data/stock_listing_v3.json', orient='records', date_format='iso')
    print('File saved successfully!')
else:
    print('File already exists!')


File already exists!


# Adding Current Stock to Item Listing

In [284]:
item_listing_df1 = pd.read_json('./data/exported_listings_v3.json')
item_listing_df1

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67c01bc13619256e0fee3511,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67c01bc13619256e0fee3512,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67c01bc13619256e0fee3513,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67c01bc13619256e0fee3514,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67c01bc13619256e0fee3515,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
5,67c01bc13619256e0fee3516,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,C,2,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
6,67c01bc13619256e0fee3517,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,C,1,2,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
7,67c01bc13619256e0fee3518,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,C,3,15,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
8,67c01bc13619256e0fee3519,200,Gloves,Protective hand coverings used by medical staf...,Consumable,C,1000,5000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
9,67c01bc13619256e0fee351a,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,C,300,2000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [285]:
stock_listing_df3

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,expirationDate,storageLocation,supplier,manufacturer,notes
0,67c01bc13619256e0fee3511,2022-01-01,Available,2628329,22,5,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...",Hall and Sons Industries,-
1,67c01bc13619256e0fee3513,2022-01-01,Available,650745,6,0,NaN,Emergency Equipment Storage,"{'name': 'Miles Inc Medical Supplies', 'contac...","Tate, Chaney and Turner Industries",-
2,67c01bc13619256e0fee3514,2022-01-01,Available,15486,23,0,NaN,Ward Equipment Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...",Webb-Ward Industries,-
3,67c01bc13619256e0fee3516,2022-01-01,Available,314121,2,2,NaN,Cardiology Equipment Room,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Campbell PLC Industries,-
4,67c01bc13619256e0fee3517,2022-01-01,Available,40522506,1,1,NaN,Radiology Imaging Suite,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Melendez, Kim and Allen Industries",-
...,...,...,...,...,...,...,...,...,...,...,...
18643,67c01bc13619256e0fee351e,2024-12-31,Available,20,5990,680,1.893283e+12,Pharmacy Storage,"{'name': 'Wang-Foster Medical Supplies', 'cont...","Hicks, Hayden and Hall Industries",-
18644,67c01bc13619256e0fee3511,2024-12-31,Available,1852964,22,1,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...",Hall and Sons Industries,-
18645,67c01bc13619256e0fee351f,2024-12-31,Available,11,3405,430,1.893283e+12,Medical Supply Room,"{'name': 'Miles Inc Medical Supplies', 'contac...",Morris-Lam Industries,-
18646,67c01bc13619256e0fee3519,2024-12-31,Available,13,4101,708,1.893283e+12,Medical Supply Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...",Wood Inc Industries,-


In [286]:
stock_listing_latest = stock_listing_df3.copy()
latest_date_per_item = stock_listing_latest.groupby('listing')['acquisitionDate'].transform('max')
stock_listing_latest['isLatest'] = stock_listing_latest['acquisitionDate'] == latest_date_per_item
stock_listing_latest = stock_listing_latest[stock_listing_latest['isLatest'] == True]
stock_listing_latest

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,expirationDate,storageLocation,supplier,manufacturer,notes,isLatest
18630,67c01bc13619256e0fee3521,2024-12-30,Available,574,1107,383,1.830125e+12,Emergency Consumables Depot,"{'name': 'Miles Inc Medical Supplies', 'contac...",Perez PLC Industries,-,True
18631,67c01bc13619256e0fee3513,2024-12-31,Available,300543,7,0,NaN,Emergency Equipment Storage,"{'name': 'Miles Inc Medical Supplies', 'contac...","Tate, Chaney and Turner Industries",-,True
18632,67c01bc13619256e0fee3520,2024-12-31,Available,820,1315,56,1.739232e+12,Emergency Consumables Depot,"{'name': 'Miles Inc Medical Supplies', 'contac...",Wolf Group Industries,-,True
18633,67c01bc13619256e0fee3516,2024-12-31,Available,225537,10,0,NaN,Cardiology Equipment Room,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Campbell PLC Industries,-,True
18634,67c01bc13619256e0fee3515,2024-12-31,Available,52721,52,2,NaN,General Equipment Storage,"{'name': 'Miles Inc Medical Supplies', 'contac...","Vazquez, Wheeler and Medina Industries",-,True
18635,67c01bc13619256e0fee3518,2024-12-31,Available,159222,5,9,NaN,ICU Equipment Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Brown, Wise and Garcia Industries",-,True
18636,67c01bc13619256e0fee3517,2024-12-31,Available,28893501,2,0,NaN,Radiology Imaging Suite,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Melendez, Kim and Allen Industries",-,True
18637,67c01bc13619256e0fee351b,2024-12-31,Available,3,10311,1627,1.893283e+12,Medical Supply Room,"{'name': 'Miles Inc Medical Supplies', 'contac...","Hawkins, Castro and Foster Industries",-,True
18638,67c01bc13619256e0fee351a,2024-12-31,Available,134,867,96,1.893283e+12,Pharmacy Storage,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Johnston, Reeves and Hunter Industries",-,True
18639,67c01bc13619256e0fee3512,2024-12-31,Available,2561065,8,0,NaN,Radiology Imaging Suite,"{'name': 'Wu-Flores Medical Supplies', 'contac...","Johnson, Macias and Villanueva Industries",-,True


In [287]:
def checkStockLevelStatus(minStockLevel, maxStockLevel, threshold, totalStocks):
    if totalStocks <= (minStockLevel + threshold):
        return 'low'
    elif totalStocks <= (maxStockLevel - threshold):
        return 'moderate'
    else:
        return 'high'

item_listing_df1 = stock_listing_latest[['listing', 'quantity']].merge(item_listing_df1, left_on='listing', right_on='_id')
item_listing_df1 = item_listing_df1.drop('listing', axis=1)

item_listing_df1.columns

# Compute stockLevelStatus
item_listing_df1['threshold'] = (item_listing_df1['maxStockLevel'] - item_listing_df1['minStockLevel']) / 3
item_listing_df1['stockLevelStatus'] = item_listing_df1.apply(lambda row: checkStockLevelStatus(row['minStockLevel'], row['maxStockLevel'], row['threshold'], row['quantity']), axis=1)

# Compute stockLevelStatus
item_listing_df1['range'] = item_listing_df1['maxStockLevel'] - item_listing_df1['minStockLevel']
item_listing_df1['stockAboveMin'] = item_listing_df1['quantity'] - item_listing_df1['minStockLevel']
item_listing_df1['stockLevelPercentage'] = np.where(
    item_listing_df1['range'] <= 0,
    0,
    np.clip((item_listing_df1['stockAboveMin'] / item_listing_df1['range']) * 100, 0, 100)
)

# Remove unnecessary columns
item_listing_df1.drop(columns=['threshold', 'range', 'stockAboveMin', 'createdAt', '_id'], inplace=True)

# Rename columns
item_listing_df1.rename(columns={'quantity': 'totalStocks'}, inplace=True)
item_listing_df1 = item_listing_df1[['itemCode', 'title', 'description', 'category', 'abcCategory', 'minStockLevel', 'maxStockLevel', 'createdBy', 'status', 'totalStocks', 'stockLevelStatus', 'stockLevelPercentage']]

item_listing_df1

,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,totalStocks,stockLevelStatus,stockLevelPercentage
0,208,Antiseptic Solution,A liquid disinfectant used to cleanse wounds a...,Consumable,C,500,3000,67b1fdb293bfc8e17f896b31,active,1107,low,24.280000
1,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,7,high,85.714286
2,207,Blood Bags,Special containers designed for the collection...,Consumable,C,300,1500,67b1fdb293bfc8e17f896b31,active,1315,high,84.583333
3,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,C,2,10,67b1fdb293bfc8e17f896b31,active,10,high,100.000000
4,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,52,low,21.600000
5,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,C,3,15,67b1fdb293bfc8e17f896b31,active,5,low,16.666667
6,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,C,1,2,67b1fdb293bfc8e17f896b31,active,2,high,100.000000
7,202,Surgical Mask,A disposable face mask that protects against a...,Consumable,C,3000,15000,67b1fdb293bfc8e17f896b31,active,10311,moderate,60.925000
8,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,C,300,2000,67b1fdb293bfc8e17f896b31,active,867,moderate,33.352941
9,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,8,high,77.777778


In [291]:
# add duplicate totalStocks column called 'currentStock'
item_listing_df1['currentStock'] = item_listing_df1['totalStocks']

# also add random date columns for 'lastABCUpdate', 'lastStockUpdate', and default 0 value for the column '__v'
item_listing_df1['lastABCUpdate'] = datetime.today().date()
item_listing_df1['lastStockUpdate'] = datetime.today().date()
item_listing_df1['__v'] = 0


In [293]:
# Export exported_listings_v4.json (to export to database)
if not os.path.exists('./data/exported_listings_v4.json'):
    item_listing_df1.to_json('./data/exported_listings_v4.json', orient='records', date_format='iso')
    print('File saved successfully!')
else:
    print('File already exists!')


File saved successfully!


# Update Stock Listing with New Listing IDs

In [314]:
stock_listing_final = pd.read_json('./data/stock_listing_v2_1.json')
stock_listing_final

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,vendor_name,vendor_contact,expiration_date,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room


In [315]:
# Rename ALL columns from snake case to camel case
stock_listing_final = stock_listing_final.rename(columns={
    'date': 'acquisitionDate',
    '_id': 'listing',
    'current_stock': 'quantity',
    'avg_usage_per_day': 'avgUsagePerDay',
    'stock_status': 'status',
    'unit_cost': 'unitCost',
    'vendor_name': 'supplierName',
    'vendor_contact': 'supplierContact',
    'expiration_date': 'expirationDate',
    }
)

In [316]:
old_item_listing = pd.read_json('./data/exported_listings.json')
old_item_listing

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67b5d1d3965831513a85d8be,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67b5d1d3965831513a85d8bf,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67b5d1d3965831513a85d8c0,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67b5d1d3965831513a85d8c1,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67b5d1d3965831513a85d8c2,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
5,67b5d1d3965831513a85d8c3,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,C,2,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
6,67b5d1d3965831513a85d8c4,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,C,1,2,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
7,67b5d1d3965831513a85d8c5,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,C,3,15,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
8,67b5d1d3965831513a85d8c6,200,Gloves,Protective hand coverings used by medical staf...,Consumable,C,1000,5000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
9,67b5d1d3965831513a85d8c7,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,C,300,2000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [317]:
stock_listing_final = stock_listing_final.merge(old_item_listing[['_id', 'title']], left_on='listing', right_on='_id')
stock_listing_final.drop('_id', axis=1, inplace=True)
stock_listing_final

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,supplierName,supplierContact,expirationDate,storageLocation,title
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room,Ventilator
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage,Defibrilator
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room,Wheelchair
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room,ECG Machine
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite,MRI Scanner
...,...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage,Syringe
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room,Ventilator
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room,Bandages
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room,Gloves


In [318]:
new_item_listing = pd.read_json('./data/exported_listings_v4_1.json')
new_item_listing.head()

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,totalStocks,stockLevelStatus,stockLevelPercentage,currentStock,lastABCUpdate,lastStockUpdate,__v
0,67c1e5046638282b88338171,208,Antiseptic Solution,A liquid disinfectant used to cleanse wounds a...,Consumable,C,500,3000,67b1fdb293bfc8e17f896b31,active,1107,low,24.280000,1107,2025-02-28T00:00:00.000,2025-02-28T00:00:00.000,0
1,67c1e5046638282b88338172,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,7,high,85.714286,7,2025-02-28T00:00:00.000,2025-02-28T00:00:00.000,0
2,67c1e5046638282b88338173,207,Blood Bags,Special containers designed for the collection...,Consumable,C,300,1500,67b1fdb293bfc8e17f896b31,active,1315,high,84.583333,1315,2025-02-28T00:00:00.000,2025-02-28T00:00:00.000,0
3,67c1e5046638282b88338174,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,C,2,10,67b1fdb293bfc8e17f896b31,active,10,high,100.000000,10,2025-02-28T00:00:00.000,2025-02-28T00:00:00.000,0
4,67c1e5046638282b88338175,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,52,low,21.600000,52,2025-02-28T00:00:00.000,2025-02-28T00:00:00.000,0


In [319]:
new_item_map = new_item_listing.set_index('title')['_id'].to_dict()
new_item_map

{'Antiseptic Solution': '67c1e5046638282b88338171',
 'Defibrilator': '67c1e5046638282b88338172',
 'Blood Bags': '67c1e5046638282b88338173',
 'ECG Machine': '67c1e5046638282b88338174',
 'Hospital Bed': '67c1e5046638282b88338175',
 'Infusion Pump': '67c1e5046638282b88338176',
 'MRI Scanner': '67c1e5046638282b88338177',
 'Surgical Mask': '67c1e5046638282b88338178',
 'IV Drip': '67c1e5046638282b88338179',
 'X-Ray-Machine': '67c1e5046638282b8833817a',
 'Wheelchair': '67c1e5046638282b8833817b',
 'Gown': '67c1e5046638282b8833817c',
 'Face Shield': '67c1e5046638282b8833817d',
 'Syringe': '67c1e5046638282b8833817e',
 'Ventilator': '67c1e5046638282b8833817f',
 'Bandages': '67c1e5046638282b88338180',
 'Gloves': '67c1e5046638282b88338181',
 'Cotton Rolls': '67c1e5046638282b88338182'}

In [320]:
# Update _id column of stock listing
stock_listing_final['listing'] = stock_listing_final['title'].map(lambda x: new_item_map[x])
stock_listing_final.drop('title', axis=1, inplace=True)
stock_listing_final

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,supplierName,supplierContact,expirationDate,storageLocation
0,67c1e5046638282b8833817f,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67c1e5046638282b88338172,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67c1e5046638282b8833817b,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67c1e5046638282b88338174,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67c1e5046638282b88338177,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...
18643,67c1e5046638282b8833817e,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage
18644,67c1e5046638282b8833817f,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room
18645,67c1e5046638282b88338180,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room
18646,67c1e5046638282b88338181,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room


In [321]:
fake = Faker()
item_listing_dic = {}

def add_supplier_data(stock_data):
    listing_id = stock_data['listing']
    if listing_id not in item_listing_dic:
        supplier_name = stock_data['supplierName']
        contact_person = fake.name()
        contact_number = stock_data['supplierContact']
        email = f'{contact_person.lower().replace(' ', '_')}@gmail.com'
        manufacturer = fake.company() + ' Industries'
        
        item_listing_dic[listing_id] = {
            'supplier': {
                'name': supplier_name,
                'contactPerson': contact_person,
                'contactNumber': contact_number,
                'email': email,
            },
            'manufacturer': manufacturer,
        }

# Add manufacturerName, contactPerson, email (supplier)
stock_listing_final.apply(add_supplier_data, axis=1)
stock_listing_final['supplier'] = stock_listing_final['listing'].map(lambda x: item_listing_dic[x]['supplier'])
stock_listing_final['manufacturer'] = stock_listing_final['listing'].map(lambda x: item_listing_dic[x]['manufacturer'])
stock_listing_final.drop(columns=['supplierName', 'supplierContact'], inplace=True)

# Add 'note' column
stock_listing_final['notes'] = '-'
stock_listing_final


,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,expirationDate,storageLocation,supplier,manufacturer,notes
0,67c1e5046638282b8833817f,2022-01-01,Available,2628329,22,5,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...","Miller, Ford and Willis Industries",-
1,67c1e5046638282b88338172,2022-01-01,Available,650745,6,0,NaN,Emergency Equipment Storage,"{'name': 'Miles Inc Medical Supplies', 'contac...","Valdez, Schmidt and Johnson Industries",-
2,67c1e5046638282b8833817b,2022-01-01,Available,15486,23,0,NaN,Ward Equipment Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Zhang, Miller and Summers Industries",-
3,67c1e5046638282b88338174,2022-01-01,Available,314121,2,2,NaN,Cardiology Equipment Room,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Turner-Spence Industries,-
4,67c1e5046638282b88338177,2022-01-01,Available,40522506,1,1,NaN,Radiology Imaging Suite,"{'name': 'Khan, Johnston and Nguyen Medical Su...",Walker Ltd Industries,-
...,...,...,...,...,...,...,...,...,...,...,...
18643,67c1e5046638282b8833817e,2024-12-31,Available,20,5990,680,1.893283e+12,Pharmacy Storage,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Patrick and Sons Industries,-
18644,67c1e5046638282b8833817f,2024-12-31,Available,1852964,22,1,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...","Miller, Ford and Willis Industries",-
18645,67c1e5046638282b88338180,2024-12-31,Available,11,3405,430,1.893283e+12,Medical Supply Room,"{'name': 'Miles Inc Medical Supplies', 'contac...","Horton, Young and Wong Industries",-
18646,67c1e5046638282b88338181,2024-12-31,Available,13,4101,708,1.893283e+12,Medical Supply Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Stewart, Gonzalez and Pruitt Industries",-


# Adding isNewStock as New Column

In [322]:
stock_listing_final_1 = stock_listing_final.copy()
stock_listing_final_1

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,expirationDate,storageLocation,supplier,manufacturer,notes
0,67c1e5046638282b8833817f,2022-01-01,Available,2628329,22,5,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...","Miller, Ford and Willis Industries",-
1,67c1e5046638282b88338172,2022-01-01,Available,650745,6,0,NaN,Emergency Equipment Storage,"{'name': 'Miles Inc Medical Supplies', 'contac...","Valdez, Schmidt and Johnson Industries",-
2,67c1e5046638282b8833817b,2022-01-01,Available,15486,23,0,NaN,Ward Equipment Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Zhang, Miller and Summers Industries",-
3,67c1e5046638282b88338174,2022-01-01,Available,314121,2,2,NaN,Cardiology Equipment Room,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Turner-Spence Industries,-
4,67c1e5046638282b88338177,2022-01-01,Available,40522506,1,1,NaN,Radiology Imaging Suite,"{'name': 'Khan, Johnston and Nguyen Medical Su...",Walker Ltd Industries,-
...,...,...,...,...,...,...,...,...,...,...,...
18643,67c1e5046638282b8833817e,2024-12-31,Available,20,5990,680,1.893283e+12,Pharmacy Storage,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Patrick and Sons Industries,-
18644,67c1e5046638282b8833817f,2024-12-31,Available,1852964,22,1,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...","Miller, Ford and Willis Industries",-
18645,67c1e5046638282b88338180,2024-12-31,Available,11,3405,430,1.893283e+12,Medical Supply Room,"{'name': 'Miles Inc Medical Supplies', 'contac...","Horton, Young and Wong Industries",-
18646,67c1e5046638282b88338181,2024-12-31,Available,13,4101,708,1.893283e+12,Medical Supply Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Stewart, Gonzalez and Pruitt Industries",-


In [323]:
stock_listing_final_1['month'] = stock_listing_final_1['acquisitionDate'].dt.to_period('M')
stock_listing_final_1['stockDiff'] = stock_listing_final_1.groupby(['listing', 'month'])[['quantity']].transform(lambda x: x.diff())
stock_listing_final_1['isNewOrder'] = stock_listing_final_1.groupby(['listing', 'month'])['stockDiff'].transform(lambda x: x > 0)
stock_listing_final_1['status'] = stock_listing_final_1['status'].apply(lambda x: x.lower())
stock_listing_final_1.drop(['stockDiff', 'month'], axis=1, inplace=True)
stock_listing_final_1

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,expirationDate,storageLocation,supplier,manufacturer,notes,isNewOrder
0,67c1e5046638282b8833817f,2022-01-01,available,2628329,22,5,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...","Miller, Ford and Willis Industries",-,False
1,67c1e5046638282b88338172,2022-01-01,available,650745,6,0,NaN,Emergency Equipment Storage,"{'name': 'Miles Inc Medical Supplies', 'contac...","Valdez, Schmidt and Johnson Industries",-,False
2,67c1e5046638282b8833817b,2022-01-01,available,15486,23,0,NaN,Ward Equipment Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Zhang, Miller and Summers Industries",-,False
3,67c1e5046638282b88338174,2022-01-01,available,314121,2,2,NaN,Cardiology Equipment Room,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Turner-Spence Industries,-,False
4,67c1e5046638282b88338177,2022-01-01,available,40522506,1,1,NaN,Radiology Imaging Suite,"{'name': 'Khan, Johnston and Nguyen Medical Su...",Walker Ltd Industries,-,False
...,...,...,...,...,...,...,...,...,...,...,...,...
18643,67c1e5046638282b8833817e,2024-12-31,available,20,5990,680,1.893283e+12,Pharmacy Storage,"{'name': 'Wang-Foster Medical Supplies', 'cont...",Patrick and Sons Industries,-,True
18644,67c1e5046638282b8833817f,2024-12-31,available,1852964,22,1,NaN,ICU Equipment Room,"{'name': 'Coleman-Bailey Medical Supplies', 'c...","Miller, Ford and Willis Industries",-,True
18645,67c1e5046638282b88338180,2024-12-31,available,11,3405,430,1.893283e+12,Medical Supply Room,"{'name': 'Miles Inc Medical Supplies', 'contac...","Horton, Young and Wong Industries",-,True
18646,67c1e5046638282b88338181,2024-12-31,available,13,4101,708,1.893283e+12,Medical Supply Room,"{'name': 'Khan, Johnston and Nguyen Medical Su...","Stewart, Gonzalez and Pruitt Industries",-,True


In [324]:
stock_listing_final_2 = stock_listing_final_1.head(30)

In [326]:
# Export stock_listing_v4.json (to export to database)
if not os.path.exists('./data/stock_listing_v4.json'):
    stock_listing_final_2.to_json('./data/stock_listing_v4.json', orient='records', date_format='iso')
    print('File saved successfully!')
else:
    print('File already exists!')


File saved successfully!
